In [209]:
import random
from Hourbid import *
from Fibid import *
from blockbid import *

In [210]:
BS_bids = Blockbid()[0]
BD_bids = Blockbid()[1]
HS_bids = HourBid()[0]
HD_bids = HourBid()[1]
FS_bids = Fibid()[0]
FD_bids = Fibid()[1]

$$\left[[HS][HD][FS][FD][BS][BD] \right]$$

In [211]:
# Heuristic Algorithm for flexible bids
temp1 = []
for i in range(len(FS_bids)):
    temp1.append(random.randint(0,23))
    
temp2 = []
for i in range(len(FS_bids)):
    temp2.append(random.randint(0,23))
    
def HA_FB(temp,Hour):
    res=[0 for i in range(len(FS_bids))]
    x=np.where(np.array(temp)==Hour)[0]
    for j in x:
        res[j]=1.0
    return res

In [212]:
# Heuristic Algorithm for flexible bids


def HA_B():
    NoneChild_D=[]
    NoneChild_S=[]

    for i in range(len(BD_bids)):
        if BD_bids.isnull()['LinkID'][i] == True:
            NoneChild_D.append(i)
    for i in range(len(BS_bids)):
        if BS_bids.isnull()['LinkID'][i] == True:
            NoneChild_S.append(i)
    BD=[ 0 for i in range(len(BD_bids))]
    BS=[ 0 for i in range(len(BS_bids))]
    for i in NoneChild_D:
        BD[i] = random.randint(0,1)
    for i in NoneChild_S:
        BS[i] = random.randint(0,1)
    return BS,BD

In [213]:
def generate_population():
    population = [[] for i in range(4)]

    # Inital Supply side population
    for i in range(24):
        length_H = HIndex_S[i][-1]-HIndex_S[i][0]+1
        individual_H = [random.randint(0, 1) for j in range(length_H)]
        population[0].append(individual_H)
        population[2].append(HA_FB(temp1,i))
    individual_B = [random.randint(0, 1) for j in range(len(BS_bids))]
    
    
    # Inital Demand side population
    for i in range(24):
        length_H = HIndex_D[i][-1]-HIndex_D[i][0]+1
        individual_H = [random.randint(0, 1) for j in range(length_H)]
        population[1].append(individual_H)
        population[3].append(HA_FB(temp2,i))
    population.append(HA_B()[0])
    population.append(HA_B()[1])
    
    return population